In [1]:
import os
import sys
import torch
import accimage
import numpy as np
import pandas as pd
from PIL import Image
from imageio import imread
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, set_image_backend, get_image_backend
import torch.nn.functional as F
import pickle
import train_utils
import data_utils
import os
import gc
import sys
import torch
import psutil
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn import metrics
from collections import Counter
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models, set_image_backend
set_image_backend('accimage')
import data_utils
import train_utils
root_dir_coad = '/n/mounted-data-drive/COAD/'

%reload_ext autoreload
%autoreload 2


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [3]:
sa_train, sa_val = data_utils.load_COAD_train_val_sa_pickle('/n/tcga_models/resnet18_WGD_v04_sa.pkl')
root_dir = '/n/mounted-data-drive/COAD/'
magnification = '5.0'

train_transform = train_utils.transform_train
val_transform = train_utils.transform_validation



train_set = data_utils.TCGADataset_tiled_slides(sa_train, root_dir, transform=train_transform, magnification=magnification)
train_loader = DataLoader(train_set, batch_size=256, pin_memory=True, num_workers=12)


val_set = data_utils.TCGADataset_tiled_slides(sa_val, root_dir, transform=val_transform, magnification=magnification)
val_loader = DataLoader(val_set, batch_size=256, pin_memory=True, num_workers=12)


state_dict_file = '/n/tcga_models/resnet18_WGD_v04.pt'
device = torch.device('cuda', 0)
output_shape = 1


resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Linear(2048, output_shape, bias=True)

saved_state = torch.load(state_dict_file, map_location=lambda storage, loc: storage)
resnet.load_state_dict(saved_state)
resnet.fc = Identity()
for param in resnet.parameters():
    param.requires_grad = False
resnet.cuda(device=device)

    
    
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v


slide_level_classification_layer = nn.Linear(2048,1)
slide_level_classification_layer.cuda()



e = 0
learning_rate = 1e-3
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(slide_level_classification_layer.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, min_lr=1e-8)


In [4]:
for e in range(5):
    train_utils.tcga_tiled_slides_training_loop(e, train_loader, resnet, 
                                    slide_level_classification_layer, criterion, 
                                 optimizer, pool_fn, train_set=train_set,device=device)

Epoch: 0, Step: 0, Slide Number: 13, Train Batch NLL: 0.7148
Epoch: 0, Step: 1, Slide Number: 38, Train Batch NLL: 1.2008
Epoch: 0, Step: 2, Slide Number: 48, Train Batch NLL: 0.7470
Epoch: 0, Step: 3, Slide Number: 79, Train Batch NLL: 0.8305
Epoch: 0, Step: 4, Slide Number: 118, Train Batch NLL: 0.9775
Epoch: 0, Step: 5, Slide Number: 138, Train Batch NLL: 0.6585
Epoch: 0, Step: 6, Slide Number: 180, Train Batch NLL: 0.7944
Epoch: 0, Step: 7, Slide Number: 225, Train Batch NLL: 0.6323
Epoch: 0, Step: 8, Slide Number: 292, Train Batch NLL: 0.6913
Epoch: 0, Step: 9, Slide Number: 322, Train Batch NLL: 0.6980
Epoch: 0, Step: 10, Slide Number: 323, Train Batch NLL: 0.7030
Epoch: 1, Step: 0, Slide Number: 73, Train Batch NLL: 0.6852
Epoch: 1, Step: 1, Slide Number: 90, Train Batch NLL: 0.6808
Epoch: 1, Step: 2, Slide Number: 129, Train Batch NLL: 0.7118
Epoch: 1, Step: 3, Slide Number: 175, Train Batch NLL: 0.6757
Epoch: 1, Step: 4, Slide Number: 213, Train Batch NLL: 0.6549
Epoch: 1, Ste

In [5]:
train_utils.tcga_tiled_slides_validation_loop(e, val_loader, resnet, 
                                    slide_level_classification_layer, criterion, 
                                 scheduler, pool_fn, val_set=val_set,device=device)

Epoch: 4, Step: 0, Slide Number: 81, Val Avg NLL: 0.7022, Val Accuracy: 0.65


(tensor(0.7022, device='cuda:0'), 0.6463414634146342)

In [7]:
torch.save(slide_level_classification_layer.state_dict(),'linear_classifier_5x.pt')

In [8]:
best_loss, accuracy  = 0.7022,0.6463

In [10]:
for e in range(5,10):
    train_utils.tcga_tiled_slides_training_loop(e, train_loader, resnet, 
                                    slide_level_classification_layer, criterion, 
                                 optimizer, pool_fn, train_set=train_set,device=device)
    loss, acc = train_utils.tcga_tiled_slides_validation_loop(e, val_loader, resnet, 
                                    slide_level_classification_layer, criterion, 
                                 scheduler, pool_fn, val_set=val_set,device=device)
    if acc> accuracy or best_loss> loss:
        print('Wrote model')
        torch.save(slide_level_classification_layer.state_dict(),'linear_classifier_5x.pt')
        best_loss = loss
        accuracy = acc

Epoch: 5, Step: 0, Slide Number: 34, Train Batch NLL: 0.7100
Epoch: 5, Step: 1, Slide Number: 97, Train Batch NLL: 0.9036
Epoch: 5, Step: 2, Slide Number: 147, Train Batch NLL: 0.9288
Epoch: 5, Step: 3, Slide Number: 205, Train Batch NLL: 0.6899
Epoch: 5, Step: 4, Slide Number: 227, Train Batch NLL: 0.6342
Epoch: 5, Step: 5, Slide Number: 296, Train Batch NLL: 0.6970
Epoch: 5, Step: 6, Slide Number: 323, Train Batch NLL: 0.6964
Epoch: 5, Step: 0, Slide Number: 81, Val Avg NLL: 0.7148, Val Accuracy: 0.38
Epoch: 6, Step: 0, Slide Number: 7, Train Batch NLL: 0.6478
Epoch: 6, Step: 1, Slide Number: 85, Train Batch NLL: 0.7153
Epoch: 6, Step: 2, Slide Number: 152, Train Batch NLL: 0.6828
Epoch: 6, Step: 3, Slide Number: 211, Train Batch NLL: 0.6782
Epoch: 6, Step: 4, Slide Number: 273, Train Batch NLL: 0.6603
Epoch: 6, Step: 5, Slide Number: 290, Train Batch NLL: 0.6363
Epoch: 6, Step: 6, Slide Number: 317, Train Batch NLL: 0.6849
Epoch: 6, Step: 7, Slide Number: 323, Train Batch NLL: 0.625